In [5]:
import dataiku
import dataikuapi
from dataiku import pandasutils as pdu
import pandas as pd

In [3]:
design_host = dataiku.api_client().host 
print(design_host)

http://127.0.0.1:20001


In [6]:
design_client = dataikuapi.DSSClient(design_host, "FJk7Jffd8TStmT4Zvx9efC6G5mYZusec") 

In [7]:
project = design_client.get_project("DSPIPELINEFRAUD")

In [11]:
project_permissions = project.get_permissions().get("permissions")

In [12]:
[permission["group"] for permission in project_permissions if permission.get("writeProjectContent", False)==True]

KeyError: 'group'

In [13]:
for permission in project_permissions: 
    print(permission)

{'group': 'MLOps', 'admin': False, 'readProjectContent': False, 'writeProjectContent': True, 'exportDatasetsData': True, 'readDashboards': False, 'writeDashboards': True, 'moderateDashboards': True, 'shareToWorkspaces': True, 'runScenarios': True, 'manageDashboardAuthorizations': False, 'manageExposedElements': True, 'manageAdditionalDashboardUsers': True, 'executeApp': False}
{'user': 'user2', 'admin': False, 'readProjectContent': False, 'writeProjectContent': True, 'exportDatasetsData': True, 'readDashboards': False, 'writeDashboards': False, 'moderateDashboards': False, 'shareToWorkspaces': True, 'runScenarios': False, 'manageDashboardAuthorizations': False, 'manageExposedElements': False, 'manageAdditionalDashboardUsers': False, 'executeApp': False}
